In [1]:
# A1 in Rq^{n*k1}; A2 in Rq^{n*k2}
# r in Rq^k1; x in Rq^k2
# y1 $ D_sigma1^k1; y2 $ D_sigma2^k2
# c in Rq
# N = 761 NTRU Prime
# N = 1024 in Lyu18
# q = 2^32
# n = 1; k1 = 3; k2 = 3 in Lyu18
# sigma = 27000
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler

In [2]:
class signature_oracle:
    def __init__(self, Rq, N, sigma):
        self.h1 = [DiscreteGaussianDistributionPolynomialSampler(Rq, N, sigma)() for _ in range(3)]
        self.h2 = Rq(list(randrange(-1, 2) for _ in range(N)))
        
    def Sign(self, m):
        return (m, vector(self.h2*vector(self.h1)))
    def Verify(self, m, sigma):
        return (sigma==self.Sign(m))

In [3]:
# has secret r, x
class Prover:
    def __init__(self, Rq, A1, A2, k1, k2, sigma, N):
        self.honest = True
        self.Rq = Rq
        self.N = N
        self.r = []
        self.x = []
        self.A1 = A1
        self.A2 = A2
        self.y1 = [DiscreteGaussianDistributionPolynomialSampler(Rq, N, sigma)() for _ in range(k1)]
        self.y2 = [DiscreteGaussianDistributionPolynomialSampler(Rq, N, sigma)() for _ in range(k2)]
        
    def Commit(self, S):
        w = matrix(self.A1)*vector(self.y1)+matrix(self.A2)*vector(self.y2)
        return w, S.Sign(w)
    def Prove(self, S, c):
        z1 = vector(self.y1) + c*vector(self.r)
        z2 = vector(self.y2) + c*vector(self.x)
        return (z1, z2, S.Sign(z1), S.Sign(z2))

In [55]:
def SQRT(z):
    temp = 0
    for i in z:
        temp += i^2
    return int(temp)^0.5

In [59]:
class Verifier:
    def __init__(self, Rq, A1, A2, k1, k2, sigma, N):
        self.Rq = Rq
        self.N = N
        self.sigma = sigma
        self.A1 = A1
        self.A2 = A2

    def Challenge(self):
        c = DiscreteGaussianDistributionPolynomialSampler(self.Rq, self.N, self.sigma)()
        return c
    
    def Verify(self, S, t, w, c, z1, z2, sigma_w, sigma_z1, sigma_z2):
        v1 = (sigma_w == S.Sign(w))
        v2 = (sigma_z1 == S.Sign(z1))
        v3 = (sigma_z2 == S.Sign(z2))
        v4 = (SQRT(z1[0].list())<2*self.sigma*self.N^0.5)
        v5 = (SQRT(z2[0].list())<2*self.sigma*self.N^0.5)
        v6 = (matrix(self.A1)*vector(z1)+matrix(self.A2)*vector(z2)-c*vector(t) == w)
        return (v1 and v2 and v3 and v4 and v5 and v6)

In [60]:
import time
class QRZKP_Protocol:
    def __init__(self, n, k1, k2, seed): # N = 761 NTRU Prime
        set_random_seed(seed)
        current_randstate().set_seed_gp()

        q = 2^32
        N = 1024
        # q = random_prime(2^512, 1)
        #n, k1, k2 = 1, 3, 3
        sigma = 27000
        Zq = Integers(q)
        P.<a> = Zq[]
        Rq = QuotientRing(P, P.ideal(a^N+1), names='a')
        
        A1_ = [[DiscreteGaussianDistributionPolynomialSampler(Rq, N, sigma)() for _ in range(k1-n)] for __ in range(n)]
        I_n = identity_matrix(n)
        A1 = [[] for _ in range(n)]
        for i in range(n):
            for j in range(n):
                A1[i].append(I_n[i][j])
            for k in range(k1-n):
                A1[i].append(A1_[i][k])

        A2_ = [[DiscreteGaussianDistributionPolynomialSampler(Rq, N, sigma)() for _ in range(k2-n-n)] for __ in range(n)]
        Zero_n = matrix(n)
        A2 = [[] for _ in range(n)]
        for i in range(n):
            for z in range(n):
                A2[i].append(Zero_n[i][z])
            for j in range(n):
                A2[i].append(I_n[i][j])
            for k in range(k2-n-n):
                A2[i].append(A2_[i][k])
        # knowledge r, x
        r = [Rq(list(randrange(-1, 2) for _ in range(N))) for _ in range(k1)]
        x = [Rq(list(randrange(-1, 2) for _ in range(N))) for _ in range(k2)]
        t = matrix(A1)*vector(r) + matrix(A2)*vector(x)
        # t = A1*r + A2*x
        
        S = signature_oracle(Rq, N, sigma)
        #========================protocol start=========================
        prover = Prover(Rq, A1, A2, k1, k2, sigma, N)
        verifier = Verifier(Rq, A1, A2, k1, k2, sigma, N)
        
        if prover.honest:
            prover.r = r
            prover.x = x
        else:
            rf = [Rq(list(randrange(-1, 2) for _ in range(N))) for _ in range(k1)]
            xf = [Rq(list(randrange(-1, 2) for _ in range(N))) for _ in range(k2)]
            prover.r = rf
            prover.x = xf
        
        start = time.time()
        w, sigma_w = prover.Commit(S)
        end = time.time()
        self.commit_time = end - start
        
        start = time.time()
        c = verifier.Challenge()
        end = time.time()
        self.challenge_time = end - start
        
        start = time.time()
        z1, z2, sigma_z1, sigma_z2 = prover.Prove(S,c)
        end = time.time()
        self.prove_time = end - start
        
        start = time.time()
        self.result = verifier.Verify(S, t, w, c, z1, z2, sigma_w, sigma_z1, sigma_z2)
        end = time.time()
        self.verify_time = end - start

In [65]:
n, k1, k2 = 12, 30, 30
ctr = 0
iteration = 20
total_commit_time = 0
total_challenge_time = 0
total_prove_time = 0
total_verify_time = 0

while ctr < iteration:
    seed = time.time_ns()
    print("iteration : %d / %d" % (ctr+1, iteration), end="\r")

    qrzkp = QRZKP_Protocol(n, k1, k2, seed)

    if qrzkp.result:
        ctr += 1
        total_commit_time += qrzkp.commit_time*10^6
        total_challenge_time += qrzkp.challenge_time*10^6
        total_prove_time += qrzkp.prove_time*10^6
        total_verify_time += qrzkp.verify_time*10^6
    else:
        pass

print("\n-----------------------------")
print("Total commit time / Average commit time       : %.2fus / %.2fus" % (total_commit_time, total_commit_time/iteration))
print("Total challenge time / Average challenge time : %.2fus / %.2fus" % (total_challenge_time, total_challenge_time/iteration))
print("Total prove time / Average prove time         : %.2fus / %.2fus" % (total_prove_time, total_prove_time/iteration))
print("Total verify time / Average verify time       : %.2fus / %.2fus" % (total_verify_time, total_verify_time/iteration))

iteration : 20 / 20
-----------------------------
Total commit time / Average commit time       : 7939630.99us / 396981.55us
Total challenge time / Average challenge time : 6219722.03us / 310986.10us
Total prove time / Average prove time         : 2187434.20us / 109371.71us
Total verify time / Average verify time       : 8952629.57us / 447631.48us
